# Pybind11 (partial) code generator

This is an attempt for a simple code generator to generate python binding for C++ libraries using [Pybind11](https://pybind11.readthedocs.io/en/stable/).

It is completely based on the ideas and code presented in the excelent article [implementing a code generator with libclang](http://szelei.me/code-generator), [Source Code](https://github.com/sztomi/code-generator).

It is absolutely not complete or well tested and probably contains mistakes. However it uses Clang for introspection and written in Python (3) so it can be easily understood, extended and adjusted. It therefore might be useful for somebody at some point in time.
For occasional use it should be suitable for generating a basic code that will be further edited and adjusted (It basically just saves some annoying typing).

## Usage:

Given a header file the generator will use clang to build the object model of the elements defined in the file and generate a function that will bind the functions, enumerations and classes that are defined in the file.
It will not generate the code to generate the module itself (i.e. will not generate 'example.cpp').

The code is shared as a Jupyter Notebook so the user can play around with the code interactively.

To notebook will allow you to generate the binding code, (section 1-3), build the module (section 4), and test it (section 5).

## Dependencies:

* libclang-py3 (libclang python binding)
* mako (for working with code templates)
* asciitree (for presenting the AST)

In [1]:
import clang.cindex  # provided through the libclang-py3 package
from mako.template import Template

#Point the python binding to your system's libclang library file 
clang.cindex.Config.set_library_file('/usr/lib/x86_64-linux-gnu/libclang-4.0.so.1')


## An aux. function to print the AST from Clang

In [2]:

import asciitree # must be version 0.2
def dump_ast(cursor,fn):
    def node_children(node):
        return [c for c in node.get_children() if (not c.location.file is None) and (c.location.file.name == fn)]

    def print_node(node):
        text = str(node.spelling or node.displayname)
        kind = str(node.kind)[str(node.kind).index('.')+1:]
        return '{} {}'.format(kind, text)

    return asciitree.draw_tree(cursor, node_children, print_node) 



# 1. Defining the object model

In [3]:
# some clang pyhton bindings return c style strings (bytes) so names need to be converted to python strings.
def b2str(b):
    if type(b) == str:
        return b
    else:
        return str(b,'utf-8')   

# Different object types that we will want to bind

#classes
class MemberFunction(object):
    def __init__(self, cursor):
        self.name = b2str(cursor.spelling)
        
class Field(object):
    def __init__(self,cursor):
        self.name = b2str(cursor.spelling)
        
class Property(object):
    '''Assumes functions have CamelCased names that start with get and set 
    i.e. getPropName and setPropName'''
    def __init__(self,getter,setter=None):
        self.name = getter.name[3:]
        self.getter = getter
        if setter is None:
            self.readonly=True
        else:
            self.readonly=False
            self.setter = setter                
        
class Constructor(object):
    def __init__(self,cursor):
        self.arg_types=[]
        for c in cursor.get_children():
            arg_t = c.type.spelling
            self.arg_types.append(arg_t)
        
class Class(object):
    def __init__(self, cursor):
        self.name = b2str(cursor.spelling)
        self.functions = []
        self.fields =[]
        self.constructors = []
        for c in cursor.get_children():
            if c.access_specifier == clang.cindex.AccessSpecifier.PUBLIC:
                if c.kind == clang.cindex.CursorKind.CXX_METHOD:
                    f = MemberFunction(c)
                    self.functions.append(f)
                elif c.kind == clang.cindex.CursorKind.FIELD_DECL:
                    f = Field(c)
                    self.fields.append(f)
                elif c.kind == clang.cindex.CursorKind.CONSTRUCTOR:
                    f = Constructor(c)
                    self.constructors.append(f)
                    
        getters = [f for f in self.functions if f.name.startswith('get')]
        setters = [f for f in self.functions if f.name.startswith('set')]
        #remove getters and setters from functions
        self.functions = [f for f in self.functions if not(f in getters or f in setters)]
        #creates a list of properties from the getters and setters
        self.properties = []
        for getter in getters:
            # try and find a setter
            try:
                setter = [f for f in setters if f.name[3:]==getter.name[3:]][0]
            except IndexError: # only a getter exists
                p = Property(getter)
            else: #both getter and setter exist
                p = Property(getter,setter)
            self.properties.append(p)
        
# global functions and enumerations
class Function(object):
    def __init__(self, cursor):
        self.name = b2str(cursor.spelling)
        
class Enumerator(object):
    def __init__(self,cursor):
        self.name = b2str(cursor.spelling)
        
class Enum(object):
    def __init__(self,cursor):
        self.name = b2str(cursor.spelling)
        self.enumerators = []
        for c in cursor.get_children():
            if c.kind == clang.cindex.CursorKind.ENUM_CONSTANT_DECL:
                self.enumerators.append(Enumerator(c))   
                

#building object model
def build_object_model(cursor,fn):
    classes = []
    functions = []
    enums = []
    for c in cursor.get_children():
        #only export elements from the parsed file
        if c.location.file.name == fn:
            if c.kind == clang.cindex.CursorKind.CLASS_DECL:
                a_class = Class(c)
                classes.append(a_class)
            elif c.kind == clang.cindex.CursorKind.FUNCTION_DECL:
                functions.append(Function(c))
            elif c.kind == clang.cindex.CursorKind.ENUM_DECL:
                enums.append(Enum(c))
                
    return classes,functions,enums


# 2. A template to generate the code form the object model

In [4]:
bind_template="""
#include <pybind11/pybind11.h>
#include "${include_file}"

namespace py = pybind11;

void bind_${include_file[:-2]}(py::module & m)
{
% for e in enums:
    py::enum_<${e.name}>(m, "${e.name}", "")
    % for ee in e.enumerators:
        .value("${ee.name}", ${e.name}::${ee.name})
    % endfor
        .export_values();
%endfor

% for f in functions:
    m.def("${f.name}",&${f.name});
% endfor

% for c in classes:
    py::class_<${c.name}>(m,"${c.name}")
    % for cc in c.constructors:
        % if len(cc.arg_types)==0:
        .def(py::init([](){return new ${c.name}();}))
        % else:
        .def(py::init<${', '.join(cc.arg_types)}>())
        % endif
    % endfor
    % for f in c.functions:
        .def("${f.name}", &${c.name}::${f.name})
    % endfor
    % for f in c.fields:
        .def_readwrite("${f.name}", &${c.name}::${f.name})
    % endfor
    % for p in c.properties:
         % if p.readonly:
        .def_property_readonly("${p.name}", &${c.name}::${p.getter.name})
         % else:
        .def_property("${p.name}", &${c.name}::${p.getter.name}, &${c.name}::${p.setter.name})
         % endif
    % endfor
    ;
% endfor
}

"""

# 3. Generating the code

In [5]:
fn='textcomponent.h'
path=''

#parsing the header file using clang
index = clang.cindex.Index.create()
translation_unit = index.parse(path+fn, ['-x', 'c++', '-std=c++11', '-D__CODE_GENERATOR__'])

#building the object model for binding from the Clang data
classes,functions,enums = build_object_model(translation_unit.cursor,path+fn)

#creating the template and rendering it with the object model
tpl = Template(text=bind_template)

rendered = tpl.render(
             classes=classes,
             functions=functions,
             enums=enums,
             include_file=fn)

#printing the resulting file and the Clang AST 
print("Generated File")
print("==============")
print(rendered)
print("Clang generated AST")
print("===================")
print(dump_ast(translation_unit.cursor,path+fn))

Generated File

#include <pybind11/pybind11.h>
#include "textcomponent.h"

namespace py = pybind11;

void bind_textcomponent(py::module & m)
{
    py::enum_<MYENUM>(m, "MYENUM", "")
        .value("OPTION_1", MYENUM::OPTION_1)
        .value("OPTION_2", MYENUM::OPTION_2)
        .value("ALL", MYENUM::ALL)
        .export_values();

    m.def("myfunc",&myfunc);

    py::class_<TextComponent>(m,"TextComponent")
        .def(py::init([](){return new TextComponent();}))
        .def(py::init<const std::string &>())
        .def(py::init<const std::string &, int>())
        .def("some_function", &TextComponent::some_function)
        .def_readwrite("myInt", &TextComponent::myInt)
        .def_property("Text", &TextComponent::getText, &TextComponent::setText)
        .def_property_readonly("ROText", &TextComponent::getROText)
    ;
}


Clang generated AST
TRANSLATION_UNIT textcomponent.h
  +--FUNCTION_DECL myfunc
  |  +--PARM_DECL x
  +--ENUM_DECL MYENUM
  |  +--ENUM_CONSTANT_DECL OPTION_1
 

In [6]:
#If you wish you can save the generated code to a file.
OUTPUT_DIR = '.'

with open(OUTPUT_DIR + "/py_{}.cpp".format(fn[:fn.find('.h')]), "w") as f:
    f.write(rendered)

# 4. Building the module

This should work on Linux. 

You might need to point your compiler to the path of the Python.h include file (/usr/include/python3.5m/ in my case)

In [7]:
cd ../src/

/home/ranr/GoogleDrive/RanWork/code-generator/src


In [8]:
!mkdir ../build
!rm -f ../build/*.o ../build/example.so

mkdir: cannot create directory ‘../build’: File exists


In [9]:
!g++ -c -g -I/usr/include/python3.5m/ -fPIC -o ../build/textcomponent.o textcomponent.cpp
!g++ -c -g -I/usr/include/python3.5m/ -fPIC -o ../build/py_textcomponent.o py_textcomponent.cpp
!g++ -c -g -I/usr/include/python3.5m/ -fPIC -o ../build/example.o example.cpp
!g++  -o ../build/example.so -shared ../build/textcomponent.o ../build/py_textcomponent.o ../build/example.o -shared -fPIC 

# 5. Testing the module

In [10]:
cd ../build

/home/ranr/GoogleDrive/RanWork/code-generator/build


In [11]:
import example


In [12]:
example.name

'Example'

In [13]:
example.myfunc(5.1)

26

In [14]:
example.OPTION_1

MYENUM.OPTION_1

In [15]:
c=example.TextComponent("foo")

In [16]:
c.myInt #uninitialized

1396104088

In [17]:
c.myInt=5

In [18]:
c.myInt

5

In [19]:
c.ROText

"can't touch this"

In [20]:
c.ROText="won't work..."

AttributeError: can't set attribute

In [21]:
c.some_function() #output in the terrminal of the notebook server

In [22]:
c.Text

'foo'

In [23]:
c.Text="bar"

In [24]:
c.Text

'bar'

In [25]:
c1=example.TextComponent() 

In [26]:
c1.Text

''

In [27]:
c2=example.TextComponent("foobar", 6)

In [28]:
c2.Text,c2.ROText,c2.myInt

('foobar', "can't touch this", 6)